# 01 - Data Cleaning

This notebook loads and cleans the clinic revenue data from 2021-2023.

**Input**: `PatientData/TOTAL COUNT 2021-2023.xlsx`  
**Output**: `data/processed/cleaned_revenue_data.csv`

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

## 1. Load Raw Data

In [2]:
# Load the Excel file
excel_path = Path('../PatientData/TOTAL COUNT 2021-2023.xlsx')
df_raw = pd.read_excel(excel_path, sheet_name='TOTAL ', header=None)

print(f"Raw data shape: {df_raw.shape}")
df_raw.head(20)

Raw data shape: (42, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,YEAR,GP,NaN,NaN,INTERNAL MEDICINE,NaN,NaN,DENTAL,NaN,NaN,GP+DENTAL+IM,NaN,NaN,PCR PATIENT COUNT,PCR CASH,NaN,FINAL REVENUE
1,2021,PATIENTS COUNT,INSURANCE MONEY,CASH,PATIENTS COUNT,INSURANCE MONEY,CASH,PATIENTS COUNT,INSURANCE MONEY,CASH,TOTAL PATIENTS,TOTAL REVENUE,NaN,NaN,NaN,NaN,NaN
2,JAN,507,54643.98,6919,0,0,0,74,11032,6700,581,79294.98,NaN,21,647,NaN,79941.98
3,FEB,554,51363.14,8107,0,0,0,57,33527.36,13343,611,106340.5,NaN,87,2031,NaN,108371.5
4,MARCH,470,38923.15,8079,0,0,0,40,14244.44,11590,510,72836.59,NaN,253,5295,NaN,78131.59
5,APRIL,357,33027.72,6228,38,5411.85,470,98,1302,7805,493,54244.57,NaN,779,17358,NaN,71602.57
6,MAY,436,48903.4,8314,96,14285.45,423,122,5421.92,9150,654,86497.77,NaN,423,11640,NaN,98137.77
7,JUNE,640,58773.6,7408,147,19324,882,145,13687.68,10080,932,110155.28,NaN,398,12600,NaN,122755.28
8,JULY,642,50421.97,6278.5,154,19072.75,980,113,4105.68,10075,909,90933.9,NaN,479,14575,NaN,105508.9
9,AUG,644,53485.12,5166,232,30014.63,663,155,14326.96,10420,1031,114075.71,NaN,440,11975,NaN,126050.71


## 2. Parse and Clean Data

The Excel file has a complex structure:
- Row 0-1: Multi-level headers
- Rows 2-13: 2021 data (Jan-Dec)
- Rows 14-16: Empty/Year marker
- Rows 17-28: 2022 data (Jan-Dec)
- Rows 29-31: Empty/Year marker
- Rows 32-41: 2023 data (Jan-Oct)

In [3]:
# Define column mapping based on Excel structure
# Columns: 0=Month, 1=GP_Patients, 2=GP_Insurance, 3=GP_Cash,
#          4=IM_Patients, 5=IM_Insurance, 6=IM_Cash,
#          7=Dental_Patients, 8=Dental_Insurance, 9=Dental_Cash,
#          10=Total_Patients, 11=Total_Revenue,
#          13=PCR_Patients, 14=PCR_Cash, 16=Final_Revenue

column_names = {
    0: 'month_name',
    1: 'gp_patients',
    2: 'gp_insurance',
    3: 'gp_cash',
    4: 'im_patients',
    5: 'im_insurance',
    6: 'im_cash',
    7: 'dental_patients',
    8: 'dental_insurance',
    9: 'dental_cash',
    10: 'total_patients',
    11: 'total_revenue',
    13: 'pcr_patients',
    14: 'pcr_cash',
    16: 'final_revenue'
}

In [4]:
def extract_year_data(df, start_row, end_row, year):
    """Extract data for a specific year from the raw dataframe."""
    year_df = df.iloc[start_row:end_row].copy()
    year_df = year_df.rename(columns=column_names)
    year_df = year_df[list(column_names.values())]
    year_df['year'] = year
    return year_df

# Extract data for each year
df_2021 = extract_year_data(df_raw, 2, 14, 2021)   # 12 months
df_2022 = extract_year_data(df_raw, 17, 29, 2022)  # 12 months
df_2023 = extract_year_data(df_raw, 32, 42, 2023)  # 10 months (Jan-Oct)

# Combine all years
df = pd.concat([df_2021, df_2022, df_2023], ignore_index=True)
print(f"Combined shape: {df.shape}")
df.head()

Combined shape: (34, 16)


,month_name,gp_patients,gp_insurance,gp_cash,im_patients,im_insurance,im_cash,dental_patients,dental_insurance,dental_cash,total_patients,total_revenue,pcr_patients,pcr_cash,final_revenue,year
0,JAN,507,54643.98,6919,0,0,0,74,11032,6700,581,79294.98,21,647,79941.98,2021
1,FEB,554,51363.14,8107,0,0,0,57,33527.36,13343,611,106340.5,87,2031,108371.5,2021
2,MARCH,470,38923.15,8079,0,0,0,40,14244.44,11590,510,72836.59,253,5295,78131.59,2021
3,APRIL,357,33027.72,6228,38,5411.85,470,98,1302,7805,493,54244.57,779,17358,71602.57,2021
4,MAY,436,48903.4,8314,96,14285.45,423,122,5421.92,9150,654,86497.77,423,11640,98137.77,2021


In [5]:
# Standardize month names
month_mapping = {
    'JAN': 1, 'FEB': 2, 'MARCH': 3, 'MAR': 3, 'APRIL': 4, 'APR': 4,
    'MAY': 5, 'JUNE': 6, 'JULY': 7, 'AUG': 8, 'AUGUST': 8,
    'SEP': 9, 'SEPTEMBER': 9, 'OCT': 10, 'OCTOBER': 10,
    'NOV': 11, 'NOVEMBER': 11, 'DEC': 12, 'DECEMBER': 12
}

df['month'] = df['month_name'].str.upper().str.strip().map(month_mapping)
print(f"Month values: {df['month'].unique()}")

Month values: [ 1  2  3  4  5  6  7  8  9 10 11 12]


In [6]:
# Create proper datetime index
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01')
df = df.sort_values('date').reset_index(drop=True)

print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Number of months: {len(df)}")

Date range: 2021-01-01 00:00:00 to 2023-10-01 00:00:00
Number of months: 34


## 3. Handle Missing PCR Data

PCR data is marked as "NO PCR" for Apr-Oct 2023. We'll set these to 0.

In [7]:
# Check PCR data
print("PCR Patients column values:")
print(df['pcr_patients'].value_counts(dropna=False))

PCR Patients column values:
pcr_patients
NO PCR    7
182       2
224       1
13        1
22        1
66        1
41        1
107       1
30        1
64        1
73        1
47        1
111       1
21        1
87        1
736       1
513       1
193       1
502       1
474       1
440       1
479       1
398       1
423       1
779       1
253       1
388       1
Name: count, dtype: int64


In [8]:
# Convert PCR columns to numeric, handling "NO PCR" as 0
df['pcr_patients'] = pd.to_numeric(df['pcr_patients'], errors='coerce').fillna(0).astype(int)
df['pcr_cash'] = pd.to_numeric(df['pcr_cash'], errors='coerce').fillna(0)

print("After cleaning:")
print(f"PCR patients range: {df['pcr_patients'].min()} to {df['pcr_patients'].max()}")
print(f"PCR cash range: {df['pcr_cash'].min()} to {df['pcr_cash'].max()}")

After cleaning:
PCR patients range: 0 to 779
PCR cash range: 0.0 to 17795.0


In [9]:
# Convert all numeric columns
numeric_cols = ['gp_patients', 'gp_insurance', 'gp_cash',
                'im_patients', 'im_insurance', 'im_cash',
                'dental_patients', 'dental_insurance', 'dental_cash',
                'total_patients', 'total_revenue', 'final_revenue']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check for any remaining NaN values
print("Missing values:")
print(df[numeric_cols].isnull().sum())

Missing values:
gp_patients         0
gp_insurance        0
gp_cash             0
im_patients         0
im_insurance        0
im_cash             0
dental_patients     0
dental_insurance    0
dental_cash         0
total_patients      0
total_revenue       0
final_revenue       0
dtype: int64


## 4. Final Data Preparation

In [10]:
# Select and reorder columns
final_columns = [
    'date', 'year', 'month',
    'gp_patients', 'gp_insurance', 'gp_cash',
    'im_patients', 'im_insurance', 'im_cash',
    'dental_patients', 'dental_insurance', 'dental_cash',
    'total_patients', 'total_revenue',
    'pcr_patients', 'pcr_cash',
    'final_revenue'
]

df_clean = df[final_columns].copy()
df_clean.head()

,date,year,month,gp_patients,gp_insurance,gp_cash,im_patients,im_insurance,im_cash,dental_patients,dental_insurance,dental_cash,total_patients,total_revenue,pcr_patients,pcr_cash,final_revenue
0,2021-01-01,2021,1,507,54643.98,6919.0,0,0.00,0,74,11032.00,6700,581,79294.98,21,647.0,79941.98
1,2021-02-01,2021,2,554,51363.14,8107.0,0,0.00,0,57,33527.36,13343,611,106340.50,87,2031.0,108371.50
2,2021-03-01,2021,3,470,38923.15,8079.0,0,0.00,0,40,14244.44,11590,510,72836.59,253,5295.0,78131.59
3,2021-04-01,2021,4,357,33027.72,6228.0,38,5411.85,470,98,1302.00,7805,493,54244.57,779,17358.0,71602.57
4,2021-05-01,2021,5,436,48903.40,8314.0,96,14285.45,423,122,5421.92,9150,654,86497.77,423,11640.0,98137.77


In [11]:
# Summary statistics
print("\n=== Data Summary ===")
print(f"Total records: {len(df_clean)}")
print(f"Date range: {df_clean['date'].min().strftime('%Y-%m')} to {df_clean['date'].max().strftime('%Y-%m')}")
print(f"\nRevenue Statistics:")
print(f"  Mean monthly revenue: ${df_clean['final_revenue'].mean():,.2f}")
print(f"  Min monthly revenue: ${df_clean['final_revenue'].min():,.2f}")
print(f"  Max monthly revenue: ${df_clean['final_revenue'].max():,.2f}")
print(f"\nPatient Statistics:")
print(f"  Mean monthly patients: {df_clean['total_patients'].mean():,.0f}")
print(f"  Total patients (all time): {df_clean['total_patients'].sum():,}")


=== Data Summary ===
Total records: 34
Date range: 2021-01 to 2023-10

Revenue Statistics:
  Mean monthly revenue: $117,610.03
  Min monthly revenue: $62,729.86
  Max monthly revenue: $157,051.49

Patient Statistics:
  Mean monthly patients: 1,102
  Total patients (all time): 37,483


In [12]:
# View all data
df_clean

,date,year,month,gp_patients,gp_insurance,gp_cash,im_patients,im_insurance,im_cash,dental_patients,dental_insurance,dental_cash,total_patients,total_revenue,pcr_patients,pcr_cash,final_revenue
0,2021-01-01,2021,1,507,54643.98,6919.0,0,0.00,0,74,11032.00,6700,581,79294.98,21,647.0,79941.98
1,2021-02-01,2021,2,554,51363.14,8107.0,0,0.00,0,57,33527.36,13343,611,106340.50,87,2031.0,108371.50
2,2021-03-01,2021,3,470,38923.15,8079.0,0,0.00,0,40,14244.44,11590,510,72836.59,253,5295.0,78131.59
3,2021-04-01,2021,4,357,33027.72,6228.0,38,5411.85,470,98,1302.00,7805,493,54244.57,779,17358.0,71602.57
4,2021-05-01,2021,5,436,48903.40,8314.0,96,14285.45,423,122,5421.92,9150,654,86497.77,423,11640.0,98137.77
5,2021-06-01,2021,6,640,58773.60,7408.0,147,19324.00,882,145,13687.68,10080,932,110155.28,398,12600.0,122755.28
6,2021-07-01,2021,7,642,50421.97,6278.5,154,19072.75,980,113,4105.68,10075,909,90933.90,479,14575.0,105508.90
7,2021-08-01,2021,8,644,53485.12,5166.0,232,30014.63,663,155,14326.96,10420,1031,114075.71,440,11975.0,126050.71
8,2021-09-01,2021,9,676,55903.00,4925.0,233,26214.38,2672,143,19019.04,10950,1052,119683.42,474,10215.0,129898.42
9,2021-10-01,2021,10,715,58388.11,5121.0,229,26280.45,1077,155,30595.92,12050,1099,133512.48,502,11585.0,145097.48


## 5. Export Cleaned Data

In [13]:
# Save to CSV
output_path = Path('../data/processed/cleaned_revenue_data.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

df_clean.to_csv(output_path, index=False)
print(f"Saved cleaned data to: {output_path}")
print(f"Shape: {df_clean.shape}")

Saved cleaned data to: ../data/processed/cleaned_revenue_data.csv
Shape: (34, 17)


In [14]:
# Verify the saved file
df_verify = pd.read_csv(output_path)
print(f"Verification - loaded {len(df_verify)} rows")
print(f"Columns: {list(df_verify.columns)}")

Verification - loaded 34 rows
Columns: ['date', 'year', 'month', 'gp_patients', 'gp_insurance', 'gp_cash', 'im_patients', 'im_insurance', 'im_cash', 'dental_patients', 'dental_insurance', 'dental_cash', 'total_patients', 'total_revenue', 'pcr_patients', 'pcr_cash', 'final_revenue']
